# Insight and debug
In order to explore the structure of a solver and to debug eventual mistakes, a few function are provided. Three are methods of the solver class:
- `show_structures()`: print a list of the structures inside the solver
- `show_connections()`: print a list of the internal connections between the structures in the solver
- `show_free_pins()`: print a list of the free pins of the structures inside the solver with the corresponding mapping to the external pins. All the pin showed here needs to me mapped for the code to work.
- `inspect()`: goes through the full hierarchy of the solver, printing for each layer the solvers and models contained in the top solver.

In order to see them in action, some structure are needed. The ones from the guide on parametric blocks will be rebuilt and used.

In [1]:
import solver as sv

with sv.Solver(name='MZM') as MZM_bal:
    WG=sv.Waveguide(L=10.0, n=2.5)
    BM=sv.GeneralBeamSplitter(ratio=0.5)
    PS=sv.PhaseShifter(param_name='PS')
    
    bm1=BM.put()
    wg1=WG.put('a0',bm1.pin['b0'])
    wg2=WG.put('a0',bm1.pin['b1'])
    ps1 =PS.put('a0',wg1.pin['b0'],param_mapping={'PS': 'PS1'})
    ps2 =PS.put('a0',wg2.pin['b0'],param_mapping={'PS': 'PS2'})

    bm2=BM.put('a0',ps1.pin['b0'])
    sv.connect(ps2.pin['b0'],bm2.pin['a1'])
    
    sv.Pin('a0').put(bm1.pin['a0'])
    sv.Pin('a1').put(bm1.pin['a1'])
    sv.Pin('b0').put(bm2.pin['b0'])
    sv.Pin('b1').put(bm2.pin['b1'])
    
with sv.Solver(name='IQ_mod') as IQ_mod:
    BM=sv.GeneralBeamSplitter(ratio=0.5)
    PS=sv.PhaseShifter(param_name='PS')
    
    bm1=BM.put()
    mz1=MZM_bal.put('a0',bm1.pin['b0'],param_mapping={'PS1' : 'I1', 'PS2' : 'I2'})
    mz2=MZM_bal.put('a0',bm1.pin['b1'],param_mapping={'PS1' : 'Q1', 'PS2' : 'Q2'})
    ps1=PS.put('a0',mz1.pin['b1'], param_mapping={'PS' : 'I'})
    ps2=PS.put('a0',mz2.pin['b1'], param_mapping={'PS' : 'Q'})
    bm2=BM.put('a0',ps1.pin['b0'])
    sv.connect(ps2.pin['b0'],bm2.pin['a1'])
    
    sv.Pin('a0').put(bm1.pin['a0'])
    sv.Pin('a1').put(bm1.pin['a1'])
    sv.Pin('b0').put(bm2.pin['b0'])
    sv.Pin('b1').put(bm2.pin['b1'])
    
    sv.Pin('DumIn0').put(mz1.pin['a1'])
    sv.Pin('DumIn1').put(mz2.pin['a1'])
    sv.Pin('DumOut0').put(mz1.pin['b0'])
    sv.Pin('DumOut1').put(mz2.pin['b0'])

Here is an example of the output on the 'MZM_bal' solver.
### show_structures


In [2]:
MZM_bal.show_structures()

Structures and pins of solver:                 Solver of MZM (id=140107474649552))
Structure (id=140107474650056) containing Model of beam-splitter with ratio 0.5 (id=140107474649720)
Structure (id=140107474650224) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664)
Structure (id=140107474651064) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664)
Structure (id=140107474586088) containing Model object (id=140107474649832) with pins: ['a0', 'b0']
Structure (id=140107474586200) containing Model object (id=140107474649832) with pins: ['a0', 'b0']
Structure (id=140107474608256) containing Model of beam-splitter with ratio 0.5 (id=140107474649720)



### show_connections

In [3]:
MZM_bal.show_connections()

Connection of solver:                 Solver of MZM (id=140107474649552))
(Structure (id=140107474650224) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664),    a0) <--> (Structure (id=140107474650056) containing Model of beam-splitter with ratio 0.5 (id=140107474649720),    b0)
(Structure (id=140107474651064) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664),    a0) <--> (Structure (id=140107474650056) containing Model of beam-splitter with ratio 0.5 (id=140107474649720),    b1)
(Structure (id=140107474586088) containing Model object (id=140107474649832) with pins: ['a0', 'b0'],    a0) <--> (Structure (id=140107474650224) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664),    b0)
(Structure (id=140107474586200) containing Model object (id=140107474649832) with pins: ['a0', 'b0'],    a0) <--> (Structure (id=140107474651064) containing Model of waveguide of lenght 10.000 and index 2.500 (

### show_free_pins

In [4]:
MZM_bal.show_free_pins()

Free pins of solver:                 Solver of MZM (id=140107474649552))
(Structure (id=140107474650056) containing Model of beam-splitter with ratio 0.5 (id=140107474649720),    a0) -->    a0
(Structure (id=140107474650056) containing Model of beam-splitter with ratio 0.5 (id=140107474649720),    a1) -->    a1
(Structure (id=140107474608256) containing Model of beam-splitter with ratio 0.5 (id=140107474649720),    b0) -->    b0
(Structure (id=140107474608256) containing Model of beam-splitter with ratio 0.5 (id=140107474649720),    b1) -->    b1



## inspect
The `inspect` function could be quite useful to debug complex structures, since it is the only one able to navigate the full hierarchy. For example here is the output on `MZM_bal`:

In [5]:
MZM_bal.inspect()

Solver of MZM (id=140107474649552)
  Structure (id=140107474650056) containing Model of beam-splitter with ratio 0.5 (id=140107474649720)
  Structure (id=140107474650224) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664)
  Structure (id=140107474651064) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664)
  Structure (id=140107474586088) containing Model object (id=140107474649832) with pins: ['a0', 'b0']
  Structure (id=140107474586200) containing Model object (id=140107474649832) with pins: ['a0', 'b0']
  Structure (id=140107474608256) containing Model of beam-splitter with ratio 0.5 (id=140107474649720)


In [6]:
IQ_mod.inspect()

Solver of IQ_mod (id=140107474649608)
  Structure (id=140107002321272) containing Model of beam-splitter with ratio 0.5 (id=140107002320208)
  Structure (id=140106476081616) containing Solver of MZM (id=140107474649552)
    Structure (id=140107474650056) containing Model of beam-splitter with ratio 0.5 (id=140107474649720)
    Structure (id=140107474650224) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664)
    Structure (id=140107474651064) containing Model of waveguide of lenght 10.000 and index 2.500 (id=140107474649664)
    Structure (id=140107474586088) containing Model object (id=140107474649832) with pins: ['a0', 'b0']
    Structure (id=140107474586200) containing Model object (id=140107474649832) with pins: ['a0', 'b0']
    Structure (id=140107474608256) containing Model of beam-splitter with ratio 0.5 (id=140107474649720)
  Structure (id=140106476081672) containing Solver of MZM (id=140107474649552)
    Structure (id=140107474650056) containing